In [13]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [9]:
# Paths to training and validation directories
train_dir = './data/train'
val_dir = './data/val'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=5,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=5,
    class_mode='binary'
)

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=3, validation_data=val_generator)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')


Found 88 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Epoch 1/3
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 291ms/step - accuracy: 0.7027 - loss: 0.5879 - val_accuracy: 0.9688 - val_loss: 0.2170
Epoch 2/3
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 261ms/step - accuracy: 0.9175 - loss: 0.1687 - val_accuracy: 0.9688 - val_loss: 0.1710
Epoch 3/3
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 233ms/step - accuracy: 1.0000 - loss: 0.0348 - val_accuracy: 0.9688 - val_loss: 0.1725
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.9309 - loss: 0.3622
Validation Accuracy: 96.88%


In [11]:
model.save("classification_model.keras")

In [2]:
model = tf.keras.models.load_model("classification_model.keras")

In [54]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np

img_path = '/home/ianyu0219/Project/labels/True_01.png'
img = image.load_img(img_path, target_size=(64, 64))
#convert to array so that we can pass into preprocess_function
img = image.img_to_array(img)
#preprocess using the same function used for training
img = preprocess_input(img, data_format=None)
#normalize image
img = img/255.0
#transform image array to a tensor
img = np.expand_dims(img, axis=0)

model.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.98641217]], dtype=float32)

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
model = tf.keras.models.load_model("classification_model.keras")

def classify(img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img = image.img_to_array(img)
    img = preprocess_input(img, data_format=None)
    img = img/255.0
    img = np.expand_dims(img, axis=0)
    
    return model.predict(img)[0][0] > 0.5